# TensorFlow: Classification with transfer learning and fine-tuning

In [ ]:
import os
import matplotlib.pyplot as plt

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import tensorflow as tf
import tensorflow_datasets as tfds

print("TF Version: ", tf.__version__)
print("TF Eager mode: ", tf.executing_eagerly())
print("TF GPU is", "available" if tf.config.list_physical_devices("GPU") else "not available")

In [ ]:
# Set image size
IMAGE_SIZE = (150, 150)
# Set the size of batches
BATCH_SIZE = 32

# Prepare dataset

In [ ]:
(raw_train_ds, raw_val_ds), ds_info = tfds.load("cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    with_info=True,
    as_supervised=True)

In [ ]:
# Display several samples
plt.figure(figsize=(10, 10))
for i, (x, y) in enumerate(raw_train_ds.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(x.numpy().astype("uint8"))
    plt.title(ds_info.features["label"].names[y.numpy()])
    plt.axis("off")

In [ ]:
# Pre-process input data using particular method
# See https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/preprocess_input
def preprocess(image, label):
    # Resize image
    image = tf.image.resize(image, size=IMAGE_SIZE)
    # Pre-process image
    image = tf.keras.applications.inception_v3.preprocess_input(tf.cast(image, dtype=tf.float32))
    return image, label

In [ ]:
BUFFER_SIZE = 10000

train_ds = (raw_train_ds
    .map(preprocess)
    .shuffle(BUFFER_SIZE)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE))

val_ds = (raw_val_ds
    .map(preprocess)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE))

# Create model

## Create base model

* Get the architecture of existing model
* Load weights to existing model
* Freeze the layers weights
* Select last layer to chain with

In [ ]:
# Link to the file with weights of pre-trained model
MODEL_URL = ("https://storage.googleapis.com/"
             "mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5")

weights_file = tf.keras.utils.get_file(origin=MODEL_URL)

In [ ]:
IMAGE_SHAPE = IMAGE_SIZE + (3,)

# Declare model without top (Dense) layer
inception_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=IMAGE_SHAPE,
    include_top=False,
    weights=None
)

# Load weights
inception_model.load_weights(weights_file)

# Freeze the base model
inception_model.trainable = False

In [ ]:
# Print model summary
inception_model.summary()

In [ ]:
base_model = tf.keras.Model(
    inception_model.input,
    inception_model.get_layer("mixed7").output # Choose `mixed7` as the base model output layer
)

In [ ]:
print(f"Base mode output shape (batch, x, y, z): {base_model.output.shape}")

## Create final model

Create a final mode with augmentation layers.

In [ ]:
# Create a sequence from augmentation layers
aug_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.4)
])

In [ ]:
inputs = tf.keras.Input(shape=IMAGE_SHAPE)
x = aug_layers(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name="accuracy")])

In [ ]:
model.summary()

# Train model

In [ ]:
LOGS_DIR = os.path.join("logs", "tf_transfer_learning")

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=5,
        min_delta=1e-2,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=LOGS_DIR,
        histogram_freq=0,
        embeddings_freq=0,
        update_freq="epoch"
    )
]

initial_epochs = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs,
    callbacks=callbacks,
    verbose=2)

__Evaluate model__

In [ ]:
# Note: The main factor that validation metrics are better than the training metrics
#       because layers like BatchNormalization and Dropout affect accuracy during
#       training. But they turned off when calculating validation loss.

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(acc))

fig, ax = plt.subplots(1,2, figsize=(12, 6))
ax[0].plot(acc, "bo", label="Training accuracy")
ax[0].plot(val_acc, "b", label="Validation accuracy")
ax[0].set_title("Training and validation accuracy")
ax[0].set_xlabel("epochs")
ax[0].set_ylabel("accuracy")
ax[0].legend()

ax[1].plot(loss, "bo", label="Training Loss")
ax[1].plot(val_loss, 'b', label="Validation Loss")
ax[1].set_title("Training and validation loss")
ax[1].set_xlabel("epochs")
ax[1].set_ylabel("loss")
ax[1].legend()

plt.show()

# Fine tuning (optional)

Fine-tuning inception base model layers in order to slightly increase model learning rate (probably).

In [ ]:
# Unfreeze the base model
base_model.trainable = True

In [ ]:
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
    metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name="accuracy")])

In [ ]:
# Set the amount of epochs for fine-tuning
fine_tune_epochs = 10
# Calculate the total epochs
total_epochs =  initial_epochs + fine_tune_epochs
# Fine-tune the model specifying initial epochs in order to continue learning
history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=len(history.epoch),
                         validation_data=val_ds)

__Evaluate model__

In [ ]:
acc += history_fine.history["accuracy"]
val_acc += history_fine.history["val_accuracy"]
loss += history_fine.history["loss"]
val_loss += history_fine.history["val_loss"]

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label="Training Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label="Start Fine Tuning")
plt.title("Training and Validation Accuracy")

plt.subplot(2, 1, 2)
plt.plot(loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label="Start Fine Tuning")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.show()

__Samples Prediction__

In [ ]:
image_batch, label_batch = val_ds.as_numpy_iterator().next()

predictions = model.predict_on_batch(image_batch).flatten()
predictions = tf.where(predictions < 0.5, 0, 1)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow((image_batch[i] + 1) / 2)
  plt.title(ds_info.features["label"].names[predictions[i]])
  plt.axis("off")